In [1]:
from spacerocks import SpaceRock, Units, Orbfit
import numpy as np

from astropy import units as u

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
units = Units()
units.angle = 'deg'

rocks = SpaceRock(a=30, 
                  e=0.2, 
                  inc=5, 
                  node=18, 
                  arg=23, 
                  t_peri=2480292.336343085,
                  epoch=2459305.5, 
                  frame='barycentric',
                  units=units)

p, planets, sim = rocks.propagate(epochs=np.linspace(2459305.5, 2459305.5 + 2 * 365.25, 10), model=2)
obs = p.observe(obscode='807')

In [4]:
units = Units()
units.ra = u.deg
units.dec = u.deg

o = Orbfit(ra=obs.ra.deg, dec=obs.dec.deg, epoch=obs.epoch.jd, obscode=807, units=units)

In [5]:
elements, errs = o.get_elements()

In [6]:
elements, errs

({'a': 29.999505351417973,
  'e': 0.19999965990194232,
  'i': 5.000008720221469,
  'lan': 17.999905179194148,
  'aop': 23.00529164908411,
  'top': 2480292.987904245},
 {'a': 0.0033216534735220865,
  'e': 0.0002465900490184061,
  'i': 0.0003235969126326204,
  'lan': 0.007196824550353988,
  'aop': 0.08448307772452943,
  'top': 14.872989861610343})

In [8]:
rock = o.spacerock

In [13]:
rock.a, rock.e, rock.inc.deg

(<Distance [29.99950535] AU>, array([0.19999966]), array([5.00000872]))

---